In [2]:
%reload_ext autoreload

%autoreload 2

import os

import iris
import xarray as xr
import metpy 

from ml_downscaling_emulator.utils import cp_model_rotated_pole, platecarree

# Making new features from existing data

## Precip (pr)

MOOSE data only has lsrain and lssnow (in $kg m^{-2} s^{-1}$ for CPM models. Total precipitation is the sum of these two values.

In [18]:
lssnow_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-27x/rcp85/01/lssnow/day/lssnow_rcp85_land-cpm_london_2.2km-coarsened-27x_01_day_19801201-19811130.nc'
lsrain_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-27x/rcp85/01/lsrain/day/lsrain_rcp85_land-cpm_london_2.2km-coarsened-27x_01_day_19801201-19811130.nc'

# lssnow_ds = xr.open_dataset(lssnow_path)
# lsrain_ds = xr.open_dataset(lsrain_path)
mvar_ds = xr.open_mfdataset([lsrain_path, lssnow_path])


In [19]:
mvar_ds

<xarray.Dataset>
Dimensions:                     (bnds: 2, grid_latitude: 64, grid_longitude: 64, time: 360)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
  * grid_latitude               (grid_latitude) float32 -1.58 -1.56 ... -0.32
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    lsrain                      (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    grid_latitude_bnds          (grid_latitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    grid_longitude_bnds         (grid_longitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    forecast_period_bnds        (time, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    lssnow                      (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [20]:
pr_ds = mvar_ds.assign(pr=lambda x: x['lssnow']+x['lsrain'])
pr_ds['pr'] = pr_ds['pr'].assign_attrs(units=mvar_ds['lssnow'].attrs['units'], cell_methods=mvar_ds['lssnow'].attrs['cell_methods'], standard_name='pr')
pr_ds = pr_ds.drop_vars(['lssnow', 'lsrain'])
pr_ds

<xarray.Dataset>
Dimensions:                     (bnds: 2, grid_latitude: 64, grid_longitude: 64, time: 360)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
  * grid_latitude               (grid_latitude) float32 -1.58 -1.56 ... -0.32
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    grid_latitude_bnds          (grid_latitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    grid_longitude_bnds         (grid_longitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    forecast_period_bnds        (time, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    pr                          (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 64, 64), meta=np.ndarray>
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

## Vorticty

Calculated from the x and y wind components

In [2]:
from windspharm.iris import VectorWind
from spharm import Spharmt, gaussian_lats_wts

In [32]:
xwind_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-8x/rcp85/01/xwind/day/xwind_rcp85_land-cpm_london_2.2km-coarsened-8x_01_day_19801201-19811130.nc'
ywind_path = f'{os.getenv("DERIVED_DATA")}/moose/london/2.2km-coarsened-8x/rcp85/01/ywind/day/ywind_rcp85_land-cpm_london_2.2km-coarsened-8x_01_day_19801201-19811130.nc'

winds_ds = xr.open_mfdataset([xwind_path, ywind_path]).sel(pressure=850.0)
wind_cubes = iris.load([xwind_path, ywind_path], iris.Constraint(pressure=850.0))

In [83]:
wind_cubes

[<iris 'Cube' of x_wind / (m s-1) (time: 360; grid_latitude: 63; grid_longitude: 64)>,
<iris 'Cube' of y_wind / (m s-1) (time: 360; grid_latitude: 63; grid_longitude: 64)>]

In [8]:
from itertools import product
import numpy as np
pairs = product(winds_ds.grid_longitude.values, winds_ds.grid_latitude.values)
longs = [p[0] for p in pairs]
lats = [p[1] for p in pairs]

In [9]:
rp_longs, rp_lats = np.array(np.meshgrid(winds_ds.grid_longitude.values, winds_ds.grid_latitude.values)).reshape(2, -1)

In [10]:
converted = platecarree.transform_points(cp_model_rotated_pole, rp_longs, rp_lats)

In [11]:
converted.reshape(64, 63, 3)[:,:,0]

array([[-1.11911882, -1.08742739, -1.05573651, ...,  0.78201613,
         0.81370222,  0.84533876],
       [ 0.87697402, -1.11851176, -1.08680641, ...,  0.75180423,
         0.78345712,  0.81515708],
       [ 0.84680746,  0.87845657, -1.11790417, ...,  0.7215646 ,
         0.75323259,  0.78489935],
       ...,
       [-0.98404721, -0.95150028, -0.9189043 , ...,  0.96825019,
        -1.0811014 , -1.04853799],
       [-1.01597517, -0.98336324, -0.95080163, ...,  0.93731078,
         0.96981279, -1.0804609 ],
       [-1.0478828 , -1.01530529, -0.98267865, ...,  0.90634218,
         0.93886018,  0.97137679]])

In [15]:
winds_ds = winds_ds.assign_coords({"longitude": (("grid_longitude", "grid_latitude"), converted.reshape(64, 63, 3)[:,:,0])})
winds_ds = winds_ds.assign_coords({"latitude": (("grid_longitude", "grid_latitude"), converted.reshape(64, 63, 3)[:,:,0])})

In [90]:
VectorWind(wind_cubes[0], wind_cubes[1])

ValueError: equally-spaced latitudes are invalid (they may be non-global)

In [104]:
u = wind_cubes[0]
v = wind_cubes[1]

In [105]:
u.coord_dims(lon)

(2,)

In [106]:
cube = u
coord = "latitude"
lat = [c for c in cube.dim_coords if coord in c.name()][0]
lat_dim = cube.coord_dims(lat)[0]


cube = v
coord = "longitude"
lon = [c for c in cube.dim_coords if coord in c.name()][0]
lon_dim = cube.coord_dims(lon)[0]

In [107]:
lat.points[0] < lat.points[1]

True

In [108]:
from iris.util import reverse

In [109]:
u = reverse(u, lat)
v = reverse(v, lat)

cube = u
coord = "latitude"
lat = [c for c in cube.dim_coords if coord in c.name()][0]
lat_dim = cube.coord_dims(lat)[0]

In [110]:
lat.points[0] < lat.points[1]

False

In [122]:
from windspharm.iris import to3d, get_apiorder, inspect_gridtype

In [123]:
u.ndim, lat_dim, lon_dim

(3, 1, 2)

In [124]:
apiorder, reorder = get_apiorder(u.ndim, lat_dim, lon_dim)

u = u.copy()
v = v.copy()
u.transpose(apiorder)
v.transpose(apiorder)

u = to3d(u.data)
v = to3d(v.data)


In [125]:
import windspharm

In [127]:
gridtype = 'regular'

In [128]:
api = windspharm.standard.VectorWind(u,v, gridtype=gridtype)

In [129]:
vrt = api.vorticity()

In [130]:
vrt.shape

(63, 64, 360)

In [118]:
u.shape

(63, 64, 360)

In [120]:
vrt

array([[[ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        ...,
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04]],

       [[ 1.75989735e-05,  2.88193169e-05,  1.83984630e-05, ...,
          1.77846887e-05,  1.02034383e-05,  5.06835422e-05],
        [ 1.70726325e-05,  2.86672675e-05,  1.84219043e-05, ...,
          1.82568547e-05,  1.02279810e

In [131]:
winds_ds

<xarray.Dataset>
Dimensions:                     (time: 360, grid_latitude: 63, grid_longitude: 64, bnds: 2)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
    pressure                    float32 850.0
  * grid_latitude               (grid_latitude) float32 -1.57 -1.55 ... -0.33
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
    longititude                 (grid_longitude, grid_latitude) float64 -1.11...
    latitude                    (grid_longitude, grid_latitude) float64 -1.11...
Dimensions without coordinates: bnds
Data variables:
    xwind                       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 63, 64), meta=np.ndarray>
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    grid_latitude_bnds          (grid_latitude, bnds) float32 dask.array<chunksize=(63, 2), meta=np.ndarray>
    grid_longitude_bnds         (grid_longitude, bnds) float32 dask.array<chunksize=(64, 2), meta=np.ndarray>
    forecast_period_bnds        (time, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    ywind                       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 63, 64), meta=np.ndarray>
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [156]:
u = winds_ds.xwind
v = winds_ds.ywind

lat = u.coords['grid_latitude']
lat_dim = u.dims.index(lat.name)

lon = u.coords['grid_longitude']
lon_dim = u.dims.index(lon.name)

In [157]:
def _reverse(array, dim):
    """Reverse an `xarray.DataArray` along a given dimension."""
    slicers = [slice(0, None)] * array.ndim
    slicers[dim] = slice(-1, None, -1)
    return array[tuple(slicers)]


In [158]:
lat.values[0] < lat.values[1]

True

In [159]:
u = _reverse(u, lat_dim)
v = _reverse(v, lat_dim)

In [160]:
lat = u.coords['grid_latitude']
lat_dim = u.dims.index(lat.name)

In [170]:
apiorder, _ = get_apiorder(u.ndim, lat_dim, lon_dim)
apiorder = [u.dims[i] for i in apiorder]
apiorder

['grid_latitude', 'grid_longitude', 'time']

In [171]:
u = u.copy().transpose(*apiorder)
v = v.copy().transpose(*apiorder)

In [172]:
u = to3d(u.values)
v = to3d(v.values)

In [174]:
api = windspharm.standard.VectorWind(u, v, gridtype=gridtype)

In [177]:
api.vorticity()

array([[[ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        ...,
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04],
        [ 1.97285954e-05,  6.24165987e-05,  4.37694798e-05, ...,
          4.66027741e-05,  1.54042555e-05,  1.05562700e-04]],

       [[ 1.75989735e-05,  2.88193169e-05,  1.83984630e-05, ...,
          1.77846887e-05,  1.02034383e-05,  5.06835422e-05],
        [ 1.70726325e-05,  2.86672675e-05,  1.84219043e-05, ...,
          1.82568547e-05,  1.02279810e

In [1]:
import metpy

AttributeError: module 'metpy' has no attribute 'calc'

In [4]:
import metpy.calc as mpcalc

In [21]:
dx, dy = mpcalc.lat_lon_grid_deltas(winds_ds.grid_longitude.values, winds_ds.grid_latitude.values)

In [28]:
dx.shape

(63, 63)

In [35]:
mpcalc.vorticity(winds_ds.isel(time=10)['xwind'], winds_ds.isel(time=10)['ywind'], dx=dx, dy=dy)

<xarray.DataArray (grid_latitude: 63, grid_longitude: 64)>
<Quantity([[ 4.33680869e-19  4.33680869e-19  4.33680869e-19 ...  9.75781955e-19
   3.37956152e-05  1.01386846e-04]
 [-6.50521303e-19 -4.33680869e-19 -4.33680869e-19 ...  0.00000000e+00
   3.37952939e-05  1.01385882e-04]
 [-2.16840434e-19  0.00000000e+00  2.16840434e-19 ...  0.00000000e+00
   3.37949768e-05  1.01384930e-04]
 ...
 [ 2.75319965e-04  2.75319965e-04  2.75319965e-04 ...  2.32732007e-04
   2.14976892e-04  1.64816116e-04]
 [-2.16840434e-19 -2.16840434e-19 -1.62630326e-19 ... -2.71050543e-20
  -1.77550762e-05 -5.32652286e-05]
 [ 3.25260652e-19  4.33680869e-19  4.87890978e-19 ...  2.16840434e-19
  -1.77550394e-05 -5.32651183e-05]], '1 / second')>
Coordinates:
    time                     object 1980-12-11 12:00:00
    pressure                 float32 850.0
  * grid_latitude            (grid_latitude) float32 -1.57 -1.55 ... -0.33
  * grid_longitude           (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period          timedelta64[ns] dask.array<chunksize=(), meta=np.ndarray>
    forecast_reference_time  object 1979-09-01 00:00:00
    realization              int32 1100000

In [16]:
winds_ds.xwind

<xarray.DataArray 'xwind' (time: 360, grid_latitude: 63, grid_longitude: 64)>
dask.array<getitem, shape=(360, 63, 64), dtype=float32, chunksize=(360, 63, 64), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) object 1980-12-01 12:00:00 ... 1981-11-30...
    pressure                 float32 850.0
  * grid_latitude            (grid_latitude) float32 -1.57 -1.55 ... -0.33
  * grid_longitude           (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_period          (time) timedelta64[ns] dask.array<chunksize=(360,), meta=np.ndarray>
    forecast_reference_time  object 1979-09-01 00:00:00
    realization              int32 1100000
    longititude              (grid_longitude, grid_latitude) float64 -1.119 ....
    latitude                 (grid_longitude, grid_latitude) float64 -1.119 ....
    longitude                (grid_longitude, grid_latitude) float64 -1.119 ....
Attributes:
    standard_name:    x_wind
    units:            m s-1
    um_stash_source:  m01s30i201
    cell_methods:     time: mean (interval: 1 hour)

(array([[2221.84151795, 2221.84151795, 2225.23364241, ..., 2225.23364241,
         2221.84151795, 2221.84151795],
        [2221.8626391 , 2221.8626391 , 2225.2547958 , ..., 2225.2547958 ,
         2221.8626391 , 2221.8626391 ],
        [2221.88348952, 2221.88348952, 2225.27567805, ..., 2225.27567805,
         2221.88348952, 2221.88348952],
        ...,
        [2222.62952985, 2222.62952985, 2226.02285738, ..., 2226.02285738,
         2222.62952985, 2222.62952985],
        [2222.63440402, 2222.63440402, 2226.02773899, ..., 2226.02773899,
         2222.63440402, 2222.63440402],
        [2222.63900736, 2222.63900736, 2226.03234935, ..., 2226.03234935,
         2222.63900736, 2222.63900736]]) <Unit('meter')>,
 array([[2223.89536482, 2223.89536482, 2223.89536482, ..., 2223.89536482,
         2223.89536482, 2223.89536482],
        [2223.89536482, 2223.89536482, 2223.89536482, ..., 2223.89536482,
         2223.89536482, 2223.89536482],
        [2223.89536482, 2223.89536482, 2223.89536482, ...